**Fully working Gradio GUI - Gemini**

In [1]:
!pip install -q google-generativeai gradio pillow

In [2]:
import gradio as gr
from google import genai
from google.genai import types
from PIL import Image
import io

# ===========================
# SETUP GEMINI CLIENT
# ===========================
API_KEY = "AIzaSyB8gUBr7CVGHViXVIfXBOku8nGd7apimodifieddjcY4I"
client = genai.Client(api_key=API_KEY)

# ===========================
# IMAGE RESIZE FUNCTION
# ===========================
def resize_image(image, max_size=1024):
    if max(image.size) <= max_size:
        return image
    ratio = max_size / max(image.size)
    new_size = (int(image.size[0] * ratio), int(image.size[1] * ratio))
    return image.resize(new_size, Image.LANCZOS)

# ===========================
# GEMINI Q&A FUNCTION
# ===========================
def ask_gemini(image, question):
    img = resize_image(image)

    img_bytes = io.BytesIO()
    img.save(img_bytes, format="JPEG")
    img_bytes = img_bytes.getvalue()

    contents = [
        types.Part.from_bytes(data=img_bytes, mime_type="image/jpeg"),
        types.Part.from_text(text=question)
    ]

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=contents
    )

    # Limit response to 4-5 lines
    lines = response.text.split('\n')
    return '\n'.join(lines[:5])

# ===========================
# CONVERSATION MEMORY
# ===========================
conversation_history = []

def add_to_memory(role, message):
    conversation_history.append({"role": role, "content": message})

def build_context():
    context_text = ""
    for turn in conversation_history:
        context_text += f"{turn['role'].capitalize()}: {turn['content']}\n"
    return context_text

# ===========================
# SUMMARIZATION FUNCTION
# ===========================
def summarize_conversation():
    context_text = build_context()
    prompt = f"Summarize the following medical conversation in concise clinical language:\n{context_text}"

    contents = [types.Part.from_text(text=prompt)]
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=contents
    )
    lines = response.text.split('\n')
    return '\n'.join(lines[:5])  # limit to 4-5 lines

# ===========================
# DIAGNOSTIC / TREATMENT FUNCTION
# ===========================
def suggest_diagnosis_treatment():
    context_text = build_context()
    prompt = f"Based on the following medical conversation, provide possible diagnostics and treatment suggestions in paragraph form:\n{context_text}"

    contents = [types.Part.from_text(text=prompt)]
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=contents
    )
    return response.text  # full paragraph

# ===========================
# MULTI-TURN CHAT FUNCTION
# ===========================
def medical_chat(image, user_message):
    add_to_memory("user", user_message)
    gemini_response = ask_gemini(image, user_message)
    add_to_memory("assistant", gemini_response)
    return gemini_response



In [4]:
# ===========================
# GRADIO INTERFACE
# ===========================
with gr.Blocks() as demo:
    gr.Markdown("## Multi-turn Medical Visual Chatbot-VLM-gemini 🤖🩺")

    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="Upload Medical Scan")
            user_input = gr.Textbox(lines=2, placeholder="Ask your question here...", label="Question")
            send_btn = gr.Button("Send")
        with gr.Column():
            chat_output = gr.Textbox(label="Assistant Answer", interactive=False, lines=10)
            summarize_btn = gr.Button("Summarize Conversation")
            summary_output = gr.Textbox(label="Conversation Summary", interactive=False, lines=10)
            diagnosis_btn = gr.Button("Suggest Diagnosis/Treatment")
            diagnosis_output = gr.Textbox(label="Diagnostics & Treatment", interactive=False, lines=15)

    send_btn.click(fn=medical_chat, inputs=[image_input, user_input], outputs=chat_output)
    summarize_btn.click(fn=summarize_conversation, inputs=[], outputs=summary_output)
    diagnosis_btn.click(fn=suggest_diagnosis_treatment, inputs=[], outputs=diagnosis_output)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a64a8c41249b21bdcb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
